In [49]:
import pandas as pd
import os
import csv

### Global Variables

In [48]:
# File paths
Path_Apl_France_Medecins = "APL_France_Medecins.csv"
Path_Distance_Medecins_Communes = "communes_distance_et_evolution_5_ans.csv"
Path_PS_Libre_Acces = "../data/external/PS_LibreAcces/PS_LibreAcces_Personne_activite.txt"

### Création des DataFrames

In [ ]:
PS_df = pd.read_csv(Path_PS_Libre_Acces, dtype='str',encoding='UTF-8', sep="|")

In [19]:
APL_France_Medecins_df = pd.read_csv(Path_Apl_France_Medecins, dtype='str',encoding='UTF-8', sep=",")

In [20]:
Distance_Medecins_Communes_df = pd.read_csv(Path_Distance_Medecins_Communes, dtype='str',encoding='UTF-8', sep=";")

In [21]:
APL_France_Medecins_df.head()

,Code commune INSEE,Communes,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2014 pour la médecine générale,Nom d'exercice
0,01001,L' Abergement-Clémenciat,2.305,2.021,765.2604758149912,[nan]
1,01002,L' Abergement-de-Varey,2.773,2.683,235.4996212788099,[nan]
2,01004,Ambérieu-en-Bugey,4.428,4.325,13638.75838414545,"['ERRARD', 'CELLIER', 'BILLANDON', 'BELEY', 'G..."
3,01005,Ambérieux-en-Dombes,4.442,4.442,1571.2112473180186,"['PIGNARD', 'DURAFFOURG']"
4,01006,Ambléon,1.084,0.936,119.10786904950405,[nan]


In [22]:
Distance_Medecins_Communes_df.head()

,Code,Nom Commune,Distance au plus proche généraliste 2017,Evolution du nbre de géné sur 5 ans 2017
0,01001,L'Abergement-Clémenciat,5,0
1,01002,L'Abergement-de-Varey,4,0
2,01004,Ambérieu-en-Bugey,0,-1
3,01005,Ambérieux-en-Dombes,0,0
4,01006,Ambléon,11,0


### Jointures des tables

In [23]:
# Renommage des colonnes (facultatif)

Distance_Medecins_Communes_df = Distance_Medecins_Communes_df.rename(columns={ Distance_Medecins_Communes_df.columns[0]: "Code INSEE" })
APL_France_Medecins_df = APL_France_Medecins_df.rename(columns={ APL_France_Medecins_df.columns[0]: "Code INSEE" })

In [24]:
result_df = pd.merge(APL_France_Medecins_df,Distance_Medecins_Communes_df[['Distance au plus proche généraliste 2017','Evolution du nbre de géné sur 5 ans 2017', 'Code INSEE']], how = 'left', on = 'Code INSEE')

In [25]:
result_df.head(5)

,Code INSEE,Communes,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2014 pour la médecine générale,Nom d'exercice,Distance au plus proche généraliste 2017,Evolution du nbre de géné sur 5 ans 2017
0,01001,L' Abergement-Clémenciat,2.305,2.021,765.2604758149912,[nan],5,0
1,01002,L' Abergement-de-Varey,2.773,2.683,235.4996212788099,[nan],4,0
2,01004,Ambérieu-en-Bugey,4.428,4.325,13638.75838414545,"['ERRARD', 'CELLIER', 'BILLANDON', 'BELEY', 'G...",0,-1
3,01005,Ambérieux-en-Dombes,4.442,4.442,1571.2112473180186,"['PIGNARD', 'DURAFFOURG']",0,0
4,01006,Ambléon,1.084,0.936,119.10786904950405,[nan],11,0


### Nettoyage du fichier

In [26]:
# On passe la variable de la population en format numérique
result_df['Population standardisée 2014 pour la médecine générale'] = result_df['Population standardisée 2014 pour la médecine générale'].apply(pd.to_numeric, errors='coerce')

In [27]:
# On arrondit la variable à l'unité près, et sans le "0." ensuite.
result_df['Population standardisée 2014 pour la médecine générale'] = result_df['Population standardisée 2014 pour la médecine générale'].apply(lambda x: round(x))

### Stastistiques basiques

In [28]:
# Statistiques de l'évolution des médecins
result_df['Evolution du nbre de géné sur 5 ans 2017'].value_counts()

0      26315
-1      1738
1       1383
-2       433
2        323
-3       159
3        117
-4        77
4         39
-5        34
-6        31
-7        24
5         21
-9        18
7         12
-8         9
-14        7
-12        7
-11        5
-10        5
6          5
-15        3
-16        3
-17        3
12         2
8          2
-23        2
9          2
10         1
-18        1
-13        1
16         1
22         1
-30        1
38         1
11         1
-21        1
-20        1
-39        1
-67        1
32         1
66         1
-31        1
-19        1
Name: Evolution du nbre de géné sur 5 ans 2017, dtype: int64

In [29]:
# Afficher les communes où l'APL = 0
result_df.loc[result_df['APL aux médecins généralistes de moins de 65 ans'] == 0]

,Code INSEE,Communes,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2014 pour la médecine générale,Nom d'exercice,Distance au plus proche généraliste 2017,Evolution du nbre de géné sur 5 ans 2017


In [30]:
# statistiques de base sur l'APL
result_df['APL aux médecins généralistes de moins de 65 ans'].describe()

count     35904
unique     5561
top           0
freq        477
Name: APL aux médecins généralistes de moins de 65 ans, dtype: object

### Création de nouvelles colonnes

#### APL Standardisé

On modifie toutes les valeurs inférieures ou égal à 1 pour qu'elles soient égales à 1. Quant aux valeurs supérieures ou égales à 5 on les fixe à 5.

In [31]:
def standardisation(x):
    if x <= 1:
        return 1
    elif x >= 5:
        return 5
    else:
        return x

In [33]:
# On passe la variable en format numérique
result_df['APL aux médecins généralistes de moins de 65 ans'] = result_df['APL aux médecins généralistes de moins de 65 ans'].apply(pd.to_numeric, errors='coerce')

In [34]:
result_df['APL 65 ans Standardisé'] = result_df['APL aux médecins généralistes de moins de 65 ans'].apply(lambda x: standardisation(x))

#### Inverse de l'APL

In [35]:
def inverse(x):
    if x != 0:
        return 1/x
    else:
        return x
    

In [36]:
result_df['inverse APL'] = result_df['APL 65 ans Standardisé'].apply(lambda x: inverse(x))

#### Population

Si la population est comprise entre 400 et 3000, Indice pop = 0.5.

In [37]:
def indice_pop(x):
    if 400 <= x <= 3000:
        return 0.5
    else:
        return 0

In [38]:
result_df['Indice pop'] = result_df['Population standardisée 2014 pour la médecine générale'].apply(lambda x: indice_pop(x))

In [39]:
result_df['Indice pop'].value_counts()

0.0    20271
0.5    15633
Name: Indice pop, dtype: int64

#### Evolution

Si l'évolution du niombre de médecins est négative, on lui assigne 0.2.

In [40]:
def evolution(x):
    if x < 0:
        return 0.2
    else:
        return 0

In [41]:
# On passe la variable en format numérique
result_df['Evolution du nbre de géné sur 5 ans 2017']= result_df['Evolution du nbre de géné sur 5 ans 2017'].apply(pd.to_numeric, errors='coerce')

In [42]:
result_df['Indice evolution'] = result_df['Evolution du nbre de géné sur 5 ans 2017'].apply(lambda x: evolution(x))

#### Ajout des Pharmacies

In [237]:
# On importe les professions de santé
PS_df = pd.read_csv(Path_PS_Libre_Acces, dtype='str',encoding='UTF-8', sep="|")

In [238]:
# On ne prend que les pharmacies d'officine
PS_df = PS_df.loc[PS_df["Libellé secteur d'activité"] == "Pharmacie d'officine"]

On créé une nouvelle colonne qui concatène la raison social avec le numéro de téléphone et l'adresse email.

In [239]:
PS_df["Téléphone (coord. structure)"] = PS_df["Téléphone (coord. structure)"].apply(str)

In [240]:
PS_df["Adresse e-mail (coord. structure)"] = PS_df["Adresse e-mail (coord. structure)"].apply(str)

In [241]:
PS_df["Raison sociale site"] = PS_df["Raison sociale site"].apply(str)

In [242]:
# Fonction pour créer une nouvelle colonne propre avec le nom de la structure et éventuellement les contacts.

def nom_et_contact(row):
    texte = row["Raison sociale site"]
    texte = texte.replace('PHARMACIE', 'Ph.')
    if row["Téléphone (coord. structure)"] != "nan" :
        texte = texte + " (" + row["Téléphone (coord. structure)"] + ")"
    if row["Adresse e-mail (coord. structure)"] != "nan" :
        texte = texte + " (" + row["Adresse e-mail (coord. structure)"] + ")"
    return texte

In [243]:
PS_df["Nom et contact"] = PS_df.apply(nom_et_contact, axis=1)

In [244]:
PS_df = PS_df.drop_duplicates(subset=["Identifiant technique de la structure"])

In [245]:
pharmacie_par_ville_df = PS_df.groupby("Code commune (coord. structure)")["Nom et contact"].apply(list)

In [246]:
pharmacie_par_ville_df = pharmacie_par_ville_df.to_frame().reset_index() # On le convertit en DF

In [251]:
pharmacie_par_ville_df = pharmacie_par_ville_df.rename(columns={ pharmacie_par_ville_df.columns[0]: "Code INSEE" })

In [252]:
pharmacie_par_ville_df.head()

,Code INSEE,Nom et contact
0,01004,"[PH. PASQUALINI (0474380226), PH. VOLLENWEIDER..."
1,01005,[PH. VINOIS-DOLIVET (0474008311)]
2,01007,[PH. BEDOY]
3,01014,[PH. GROSS (0474773819)]
4,01021,[PH. SCHINDELE (0474007732)]


##### Fusion de pharmacie avec le DF

In [255]:
result_df = pd.merge(result_df, pharmacie_par_ville_df[['Code INSEE',"Nom et contact"]],on='Code INSEE', how='left')

In [257]:
result_df  = result_df.rename(columns={ "Nom et contact": "Pharmacie" })

In [272]:
result_df["Pharmacie"] = result_df["Pharmacie"].apply(str)

In [ ]:
# Statistiques sur les pharmacies isolées
result_df.loc[(result_df["Nom d'exercice"] == "[nan]") & (result_df["Pharmacie"] != "nan")].describe()

##### Calcul du score de la phamarcie isolée

In [281]:
def ph_isolee(row):
    if (row["Nom d'exercice"] == "[nan]") & (row["Pharmacie"] != "nan"):
        return 0.2
    else:
        return 0

In [282]:
result_df["Indice Ph isolée"]  = result_df.apply(ph_isolee, axis=1)

### Calcul du score

In [286]:
#colonnes utilisées pour calculer le score
colonnes_pour_score_liste = ['inverse APL','Indice pop','Indice evolution', 'Indice Ph isolée']

In [287]:
result_df['Score'] = result_df[colonnes_pour_score_liste].sum(axis=1)

In [288]:
result_df.head(5)

,Code INSEE,Communes,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2014 pour la médecine générale,Nom d'exercice,Distance au plus proche généraliste 2017,Evolution du nbre de géné sur 5 ans 2017,APL 65 ans Standardisé,inverse APL,Indice pop,Indice evolution,Score,Pharmacie,Indice Ph isolée
0,01001,L' Abergement-Clémenciat,2.305,2.021,765,[nan],5,0.0,2.021,0.494805,0.5,0.0,0.994805,nan,0.0
1,01002,L' Abergement-de-Varey,2.773,2.683,235,[nan],4,0.0,2.683,0.372717,0.0,0.0,0.372717,nan,0.0
2,01004,Ambérieu-en-Bugey,4.428,4.325,13639,"['ERRARD', 'CELLIER', 'BILLANDON', 'BELEY', 'G...",0,-1.0,4.325,0.231214,0.0,0.2,0.431214,"['PH. PASQUALINI (0474380226)', 'PH. VOLLENWEI...",0.0
3,01005,Ambérieux-en-Dombes,4.442,4.442,1571,"['PIGNARD', 'DURAFFOURG']",0,0.0,4.442,0.225124,0.5,0.0,0.725124,['PH. VINOIS-DOLIVET (0474008311)'],0.0
4,01006,Ambléon,1.084,0.936,119,[nan],11,0.0,1.000,1.000000,0.0,0.0,1.000000,nan,0.0


### Export du fichier en CSV

In [290]:
result_df.to_csv('score_communes.csv', sep=',', encoding='utf-8', quotechar='"', index=False, quoting=csv.QUOTE_ALL)